In [114]:
import requests
import pandas as pd

# CityBikes

In [115]:
# API endpoint for Toronto city
url = 'https://api.citybik.es/v2/networks/toronto'
limit = 100

params = {'limit': limit}

# Send GET request to retrieve bike-sharing data for Toronto
response = requests.get(url, params=params)

# Extract the data from the response
data = response.json()

# Extract the list of stations
stations = data['network']['stations']

# Iterate over each station and access the details
for station in stations:
    station_id = station.get('id')
    name = station.get('name')
    latitude = station.get('latitude')
    longitude = station.get('longitude')
    num_of_bikes = station.get('free_bikes')

# Create a list to store the station details
station_details = []

# Iterate over the stations and extract the desired information
for station in stations:
    station_id = station['id']
    name = station['name']
    latitude = station['latitude']
    longitude = station['longitude']
    num_of_bikes = station['free_bikes']

    # Append the station details as a dictionary to the list
    station_details.append({
        'Station ID': station_id,
        'Name': name,
        'Latitude': latitude,
        'Longitude': longitude,
        'Number of Bikes': num_of_bikes
    })

In [116]:
# Create the bike stations DataFrame
bike_df = pd.DataFrame(station_details)
bike_df

,Station ID,Name,Latitude,Longitude,Number of Bikes
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,17
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,3
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693,23
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,3
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,7
...,...,...,...,...,...
658,b1378d745af2e60fe542370cb6374226,Lynn William St / Pirandello St,43.639125,-79.414232,0
659,c845b03b2ffaa03ebba13b5c61da3af2,Bay St / Queens Quay W (Ferry Terminal),43.640978,-79.376785,22
660,f8bbce635f366ade23994d02cb6fcc75,College St / Huron St,43.658148,-79.398167,8
661,5539f3700e4ce9e5701353a667924538,Ossington Ave / Bruce St,43.645702,-79.419332,12


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [147]:
API_key = 'fsq3dQYQp2ka71CYVbI6ehUtmigysRY/oygrq7j7KFRHyI8='
url = 'https://api.foursquare.com/v3/places/nearby'
radius = 1000  # Radius in meters
limit = 10

In [148]:
# Create a list to store the restaurant details
fq_rest_details = []

In [149]:
# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'll': f'{latitude},{longitude}',
        'radius': radius,
        'query': 'restaurant',
        'limit': limit
    }
    headers = {"Authorization": API_key}

    response = requests.get(url, params=params, headers=headers)

    fq_rest_data = response.json()

    # Extract the list of restaurants
    restaurants = fq_rest_data['results']

    # Iterate over each restaurant and extract the details
    for restaurant in restaurants:
        name = restaurant['name']
        location = restaurant['location']['formatted_address']
        latitude = restaurant['geocodes']['main']['latitude']
        longitude = restaurant['geocodes']['main']['longitude']

        # Append the restaurant details as a dictionary to the list
        fq_rest_details.append({
            'Retaurant name': name,
            'Name': name,
            'Latitude': latitude,
            'Longitude': longitude
        })


Put your parsed results into a DataFrame

In [150]:
# Create Foursquare restaurants DataFrame
fq_df = pd.DataFrame(fq_rest_details)

fq_df

,Retaurant name,Name,Latitude,Longitude
0,Gio Rana's Really Really Nice Restaurant,Gio Rana's Really Really Nice Restaurant,43.663345,-79.330418
1,Occasions Restaurant,Occasions Restaurant,43.673316,-79.318244
2,Regency Restaurant Inc,Regency Restaurant Inc,43.672004,-79.322433
3,Famous Indian Cuisine,Famous Indian Cuisine,43.672148,-79.321988
4,Restaurant Tiflisi,Restaurant Tiflisi,43.669848,-79.301657
...,...,...,...,...
5719,Nevada Restaurant,Nevada Restaurant,43.669567,-79.301659
5720,Fuse,Fuse,43.655932,-79.363890
5721,Honey Bee Restaurant,Honey Bee Restaurant,43.670525,-79.298490
5722,Bikkuri Japanese Restaurant,Bikkuri Japanese Restaurant,43.670533,-79.296807


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [154]:
API_key = 'u9nqlkJV8-tYun0YccBayBbLZ9CjblVJp0vAX9R12paPQOSv10Y0IDpDZVyfGoABFEo3CftG59CPjqESoxa3wJvgA9Ju8QyTav7LVuc9G-4QmWDcYWZmArNl3sJ7ZHYx'
url = 'https://api.yelp.com/v3/businesses/search'
radius = 1000  # Radius in meters
limit = 10

In [155]:
# Create a list to store restaurant details
y_rest_details = []

In [156]:
# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': 'restaurants',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {API_key}'
    }

    # Send API request to Yelp
    response = requests.get(url, params=params, headers=headers)
    y_rest_data = response.json()

y_rest_data


{'error': {'code': 'ACCESS_LIMIT_REACHED',
  'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}

Put your parsed results into a DataFrame

In [ ]:
# Create Yelp restaurants DataFrame
yelp_df = pd.DataFrame(y_rest_details)

yelp_df

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating